In [51]:
import pickle
import pandas as pd
import csv

results = list()

datasetFile = open("fnmr/dataset/fnmr_dataset_morph.pickle", 'rb')
dataset = pickle.load(datasetFile)

resultFile = open("fnmr/results/openface_fnmr_morph.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/arcface_fnmr_morph.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/facenet_fnmr_morph.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/vggface_fnmr_morph.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/results/faceapi_fnmr_morph.pickle", 'rb')
faceapiDataset = (pickle.load(resultFile))


resultDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for result in results:
    for algorithm in result.keys():
        if result[algorithm]:
            resultDict[algorithm] = result[algorithm]

In [52]:
labels = pd.read_csv(r'./labels_morph.csv')
names = list()
labelDict = dict()

for index, row in labels.iterrows():
    labelDict[str(row["id_num"])] = dict()
    labelDict[str(row["id_num"])]["Asian"] = False
    labelDict[str(row["id_num"])]["White"] = False
    labelDict[str(row["id_num"])]["Black"] = False
    labelDict[str(row["id_num"])]["Hispanic"] = False

    if row["race"] == "W":
        labelDict[str(row["id_num"])]["White"] = True
    elif row["race"] == "B":
        labelDict[str(row["id_num"])]["Black"] = True
    elif row["race"] == "A":
        labelDict[str(row["id_num"])]["Asian"] = True  
    elif row["race"] == "H":
        labelDict[str(row["id_num"])]["Hispanic"] = True

In [53]:
processDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for algorithm in resultDict.keys():
    for index in range(len(dataset)):
        referenceSubject = (dataset[index][0].split('_'))[0].lstrip("0")
        comparisonSubject = (dataset[index][1].split('_'))[0].lstrip("0")

        pairNumber = 'pair_' + str(index + 1)
        if not(referenceSubject in processDict[algorithm].keys()):
            processDict[algorithm][referenceSubject] = list()
        processDict[algorithm][referenceSubject].append(resultDict[algorithm][pairNumber])

faceAPIResult = dict()

for comparison in faceapiDataset:
    parsedComparison = comparison.split(" ")
    parsedComparison[0] = parsedComparison[0].replace("/home/ubuntu/Album2/", "")
    parsedComparison[1] = parsedComparison[1].replace("/home/ubuntu/Album2/", "")
    parsedComparison[2].replace("undefined", "1")
    try: 
        parsedComparison[2] = float(parsedComparison[2])
    except:
        parsedComparison[2] = float(1)

    
    referenceSubject = (parsedComparison[0].split('_'))[0].lstrip("0")
    comparisonSubject = (parsedComparison[1].split('_'))[0].lstrip("0")
    distance = parsedComparison[2]

    comparisonDict = dict()
    comparisonDict["distance"] = distance
    if distance >= float(.6):
        comparisonDict["verified"] = False
    else:
        comparisonDict["verified"] = True

    if not(referenceSubject in faceAPIResult.keys()):
        faceAPIResult[referenceSubject] = list()
    faceAPIResult[referenceSubject].append(comparisonDict)

processDict["FaceAPI"] = faceAPIResult

In [54]:
verifiedDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}


for algorithm in processDict.keys():
    for subject in processDict[algorithm].keys():
        for comparison in processDict[algorithm][subject]:
            if not(subject in verifiedDict[algorithm].keys()):
                verifiedDict[algorithm][subject] = list()
            verified = False
            if comparison["distance"] < .6:
                verified = True
            verifiedDict[algorithm][subject].append(verified)



In [55]:
verifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

for algorithm in verifiedDict.keys():
    for subject in verifiedDict[algorithm].keys():
        for comparison in verifiedDict[algorithm][subject]:
            if not(subject in verifiedStats[algorithm].keys()):
                verifiedStats[algorithm][subject] = dict()
                verifiedStats[algorithm][subject]["tpr"] = 0
                verifiedStats[algorithm][subject]["fnr"] = 0
            if comparison:
                verifiedStats[algorithm][subject]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + 1
            else:
                verifiedStats[algorithm][subject]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + 1

Overall FPR and TNR by algorithm:

In [56]:
summaryVerifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

for algorithm in verifiedStats.keys():

    summaryVerifiedStats[algorithm]["tpr"] = 0
    summaryVerifiedStats[algorithm]["fnr"] = 0

    for subject in verifiedStats[algorithm].keys():
        summaryVerifiedStats[algorithm]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + summaryVerifiedStats[algorithm]["tpr"]
        summaryVerifiedStats[algorithm]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + summaryVerifiedStats[algorithm]["fnr"]

print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + str(summaryVerifiedStats[algorithm]["fnr"]))
    print("TP:  " + str(summaryVerifiedStats[algorithm]["tpr"]))  
    print("FNMR:  " + str(round(float(summaryVerifiedStats[algorithm]["fnr"])/float(summaryVerifiedStats[algorithm]["fnr"] + summaryVerifiedStats[algorithm]["tpr"]) * 100, 3)) + "%")  


FNMR Results:

Facenet
FN:  3115
TP:  38400
FNMR:  7.503%

VGG-Face
FN:  1683
TP:  39832
FNMR:  4.054%

OpenFace
FN:  2347
TP:  39168
FNMR:  5.653%

ArcFace
FN:  4871
TP:  36644
FNMR:  11.733%

FaceAPI
FN:  275
TP:  41240
FNMR:  0.662%


In [57]:
summaryVerifiedStatsLabelled = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}


for algorithm in verifiedStats.keys():
    for label in labelDict[list(labelDict.keys())[0]].keys():
        if not(label in summaryVerifiedStatsLabelled[algorithm].keys()):
            summaryVerifiedStatsLabelled[algorithm][label] = dict()
        summaryVerifiedStatsLabelled[algorithm][label]["tpr"] = 0
        summaryVerifiedStatsLabelled[algorithm][label]["fnr"] = 0

    for subject in verifiedStats[algorithm].keys():
        labelledRace = ""
        for label in labelDict[subject].keys():
            if labelDict[subject][label]:
                labelledRace = label
        if labelledRace == "":
            continue
        summaryVerifiedStatsLabelled[algorithm][labelledRace]["tpr"] = verifiedStats[algorithm][subject]["tpr"] + summaryVerifiedStatsLabelled[algorithm][labelledRace]["tpr"]
        summaryVerifiedStatsLabelled[algorithm][labelledRace]["fnr"] = verifiedStats[algorithm][subject]["fnr"] + summaryVerifiedStatsLabelled[algorithm][labelledRace]["fnr"]


for algorithm in summaryVerifiedStatsLabelled.keys():
    for label in summaryVerifiedStatsLabelled[algorithm]:
        summaryVerifiedStatsLabelled[algorithm][label]["fnmr"] = str(round((float(summaryVerifiedStatsLabelled[algorithm][label]["fnr"])/float(summaryVerifiedStatsLabelled[algorithm][label]["fnr"] + summaryVerifiedStatsLabelled[algorithm][label]["tpr"]))*100,3)) + "%"

print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["fnr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["fnr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["fnr"]) + "\tHispanic: " + str(summaryVerifiedStatsLabelled[algorithm]["Hispanic"]["fnr"]))
    print("TP:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["tpr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["tpr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["tpr"]) + "\tHispanic: " + str(summaryVerifiedStatsLabelled[algorithm]["Hispanic"]["tpr"]))  
    print("FNMR:  " + "\tBlack: " + str(summaryVerifiedStatsLabelled[algorithm]["Black"]["fnmr"])+ "\tWhite: " + str(summaryVerifiedStatsLabelled[algorithm]["White"]["fnmr"]) + "\tAsian: " + str(summaryVerifiedStatsLabelled[algorithm]["Asian"]["fnmr"]) + "\tHispanic: " + str(summaryVerifiedStatsLabelled[algorithm]["Hispanic"]["fnmr"]))



FNMR Results:

Facenet
FN:  	Black: 2207	White: 761	Asian: 16	Hispanic: 124
TP:  	Black: 30054	White: 7107	Asian: 89	Hispanic: 1112
FNMR:  	Black: 6.841%	White: 9.672%	Asian: 15.238%	Hispanic: 10.032%

VGG-Face
FN:  	Black: 1163	White: 412	Asian: 12	Hispanic: 92
TP:  	Black: 31098	White: 7456	Asian: 93	Hispanic: 1144
FNMR:  	Black: 3.605%	White: 5.236%	Asian: 11.429%	Hispanic: 7.443%

OpenFace
FN:  	Black: 1925	White: 360	Asian: 10	Hispanic: 47
TP:  	Black: 30336	White: 7508	Asian: 95	Hispanic: 1189
FNMR:  	Black: 5.967%	White: 4.575%	Asian: 9.524%	Hispanic: 3.803%

ArcFace
FN:  	Black: 3705	White: 999	Asian: 24	Hispanic: 134
TP:  	Black: 28556	White: 6869	Asian: 81	Hispanic: 1102
FNMR:  	Black: 11.484%	White: 12.697%	Asian: 22.857%	Hispanic: 10.841%

FaceAPI
FN:  	Black: 185	White: 89	Asian: 0	Hispanic: 1
TP:  	Black: 32076	White: 7779	Asian: 105	Hispanic: 1235
FNMR:  	Black: 0.573%	White: 1.131%	Asian: 0.0%	Hispanic: 0.081%
